encoder

In [1]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Input,Flatten,Dense
from keras.utils.data_utils import get_file
import keras.backend as K
import h5py
import numpy as np
import tensorflow as tf


MEAN_PIXEL = np.array([103.939, 116.779, 123.68])



def vgg_layers(inputs, target_layer):
    
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
#     x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)


    # Block 2
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
#     x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)


    # Block 3
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
#     x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
#     x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Conv2D(1, (3, 3), activation='relu', padding='same', name='downsample')(x)
    return x


def VGG19(input_tensor=None, input_shape=None, target_layer=1):
    """
    VGG19, up to the target layer (1 for relu1_1, 2 for relu2_1, etc.)
    """
    if input_tensor is None:
        inputs = Input(shape=input_shape)
    else:
        inputs = Input(tensor=input_tensor, shape=input_shape)
    model = Model(inputs, vgg_layers(inputs, target_layer), name='vgg19')
    # load_weights(model)
    return model


def preprocess_input(x):
    # Convert 'RGB' -> 'BGR'
    if type(x) is np.ndarray:
        x = x[..., ::-1]
    else:
        x = tf.reverse(x, [-1])

    return x - MEAN_PIXEL


Using TensorFlow backend.


decoder


In [2]:
from keras.layers import Input, Conv2D, UpSampling2D

def decoder_layers(inputs, layer):

    x=decoder_block4(inputs)
    x=decoder_block3(x)
    x=decoder_block2(x)
    x=decoder_block1(x)
    return x



def decoder_block1(inputs):
    x = UpSampling2D((2, 2), name='decoder_block1_upsample')(inputs)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='decoder_block1_conv2')(x)
#     x = Conv2D(32, (3, 3), activation='relu', padding='same', name='decoder_block1_conv1')(x)
    return x

def decoder_block2(inputs):
    x = UpSampling2D((2, 2), name='decoder_block2_upsample')(inputs)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_block2_conv2')(x)
#     x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decoder_block2_conv1')(x)
    return x

def decoder_block3(inputs):
    x = UpSampling2D((2, 2), name='decoder_block3_upsample')(inputs)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_block3_conv3')(x)
#     x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decoder_block3_conv2')(x)
    return x

def decoder_block4(inputs):
    x = UpSampling2D((2, 2), name='decoder_block4_upsample')(inputs)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block4_conv3')(x)
#     x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decoder_block4_conv1')(x)
    return x



In [3]:
from keras.models import Model, Sequential, load_model
from keras.layers import Conv2D, Input
import keras.backend as K

LAMBDA=1

def l2_loss(x):
    return K.sum(K.square(x)) / 2

class EncoderDecoder:
    def __init__(self, input_shape=(256, 256, 3), target_layer=5,
                 decoder_path=None):
        self.input_shape = input_shape
        self.target_layer = target_layer

        self.encoder = VGG19(input_shape=input_shape, target_layer=target_layer)
        if decoder_path:
            self.decoder = load_model(decoder_path)
        else:
            self.decoder = self.create_decoder(target_layer)

        self.model = Sequential()
        self.model.add(self.encoder)
        self.model.add(self.decoder)

        self.loss = self.create_loss_fn(self.encoder)

        self.model.compile('adam', self.loss)

    def create_loss_fn(self, encoder):
        def get_encodings(inputs):
            encoder = VGG19(inputs, self.input_shape, self.target_layer)
            return encoder.output

        def loss(img_in, img_out):
            encoding_in = get_encodings(img_in)
            encoding_out = get_encodings(img_out)
            return l2_loss(img_out - img_in) + \
                   LAMBDA*l2_loss(encoding_out - encoding_in)
        return loss

    def create_decoder(self, target_layer):
        inputs = Input(shape=self.encoder.output_shape[1:])
        layers = decoder_layers(inputs, target_layer)
        output = Conv2D(3, (3, 3), activation='relu', padding='same',
                        name='decoder_out')(layers)
        return Model(inputs, output, name='decoder_%s' % target_layer)

    def export_decoder(self):
        self.decoder.save('decoder_%s.h5' % self.target_layer)

    def export_encoder(self):
        self.encoder.save('encoder_%s.h5' % self.target_layer)

In [4]:
import sys
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import Callback
from scipy.misc import imresize, imsave
import numpy as np



TRAIN_PATH = 'totaldata/'
TARGET_SIZE = (256, 256)
BATCH_SIZE = 8


datagen = ImageDataGenerator()
# gen = datagen.flow_from_directory(TRAIN_PATH, target_size=TARGET_SIZE,
#                                   batch_size=BATCH_SIZE, class_mode=None)


def create_gen(img_dir, target_size, batch_size):
    datagen = ImageDataGenerator()
    gen = datagen.flow_from_directory(img_dir, target_size=target_size,
                                      batch_size=batch_size, class_mode=None)

    def tuple_gen():
        for img in gen:
            if img.shape[0] != batch_size:
                continue

            # (X, y)
            yield (img, img)

    return tuple_gen()

# This needs to be in scope where model is defined
class OutputPreview(Callback):
    def __init__(self, model, test_img_path, increment, preview_dir_path):
        test_img = image.load_img(test_img_path)
        test_img = imresize(test_img, (256, 256, 3))
        test_target = image.img_to_array(test_img)
        test_target = np.expand_dims(test_target, axis=0)
        self.test_img = test_target
        self.model = model

        self.preview_dir_path = preview_dir_path

        self.increment = increment
        self.iteration = 0

    def on_batch_end(self, batch, logs={}):
        if (self.iteration % self.increment == 0):
            output_img = self.model.predict(self.test_img)[0]
            fname = '%d.jpg' % self.iteration
            out_path = os.path.join(self.preview_dir_path, fname)
            imsave(out_path, output_img)

        self.iteration += 1


gen = create_gen(TRAIN_PATH, TARGET_SIZE, BATCH_SIZE)


Found 13736 images belonging to 1 classes.


In [5]:
num_samples = 13736 
epochs = 12
steps_per_epoch = num_samples // BATCH_SIZE

target_layer = 3

encoder_decoder = EncoderDecoder(target_layer=target_layer)
encoder_decoder.encoder.summary()
encoder_decoder.decoder.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 32)      896       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 32)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 64)      18496     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block3_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [6]:


callbacks = [OutputPreview(encoder_decoder, './1.jpg', 800, 'out' )]
encoder_decoder.model.fit_generator(gen, steps_per_epoch=steps_per_epoch,
        epochs=epochs, callbacks=callbacks)
encoder_decoder.export_decoder()
encoder_decoder.export_encoder()

/conda/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Epoch 1/12
   4/1717 [..............................] - ETA: 26:11 - loss: 5459878720.0000

/conda/lib/python3.6/site-packages/ipykernel_launcher.py:57: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/conda/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.116091). Check your callbacks.
  % delta_t_median)


1717/1717 [==============================] - 98s 57ms/step - loss: 628994063.7857
Epoch 2/12
1717/1717 [==============================] - 94s 55ms/step - loss: 432643444.4636
Epoch 3/12
1717/1717 [==============================] - 95s 55ms/step - loss: 416362782.5649
Epoch 4/12
1717/1717 [==============================] - 94s 55ms/step - loss: 396266352.5498
Epoch 5/12
1717/1717 [==============================] - 94s 55ms/step - loss: 385696179.9977
Epoch 6/12
1717/1717 [==============================] - 94s 55ms/step - loss: 378967053.4188
Epoch 7/12
1717/1717 [==============================] - 94s 55ms/step - loss: 372956194.5906
Epoch 8/12
1717/1717 [==============================] - 94s 55ms/step - loss: 367196470.1223
Epoch 9/12
1717/1717 [==============================] - 94s 55ms/step - loss: 363338561.5842
Epoch 10/12
1717/1717 [==============================] - 94s 55ms/step - loss: 360235008.3541
Epoch 11/12
1717/1717 [==============================] - 94s 55ms/step - loss: 3